Estou usando esse script para o meu TCC, onde quero puxar perfis aleatórios baseados em certas palavras-chave para analisar os perfils que as tweetaram.

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import sqlite3
import time

pd.set_option('display.max_columns', 50)

chave_consumidor = 'XXXXXXXXXXXXXXXXXXXXXXXXX'            
segredo_consumidor = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
                     
token_acesso = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'             
token_acesso_segredo = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
                       
autenticacao = tweepy.OAuthHandler(chave_consumidor, segredo_consumidor)
autenticacao.set_access_token(token_acesso, token_acesso_segredo)

twitter = tweepy.API(autenticacao)

for keyword in ['XXXX', 'XXXX', '#XXXX', '#XXXX']: #digite aqui as palavras-chave desejadas
    
    print(f'Consulta {keyword}')
    
    #para cada palavra-chave serão buscados 50 tweets em português
    resultados = twitter.search(q = keyword, lang = 'pt', count = 50) 

    id_usuario = [] #id do usuário
    usuario = [] #@ do usuário
    usuario_tela = [] #nome do usuário
    data_criacao = [] #data de criação da conta
    n_seguidores = [] #número de seguidores
    n_retweets = [] #número de retweets da conta
    perfil_fechado = [] #booelano se o perfil é fechado
    total_favoritos = [] #total de curtidas da conta
    verificado = [] #booleano se o perfil é verificado
    tem_background_imagem = [] #booleano se o perfil tem imagem de capa

    for tweet in resultados:
        id_usuario.append(tweet.user.id_str)
        usuario.append(tweet.user.name)
        usuario_tela.append(tweet.user.screen_name)
        data_criacao.append(tweet.user.created_at)
        perfil_fechado.append(tweet.user.protected)
        n_seguidores.append(tweet.user.followers_count)
        n_retweets.append(tweet.retweet_count)
        total_favoritos.append(tweet.user.favourites_count)
        verificado.append(tweet.user.verified)
        tem_background_imagem.append(tweet.user.profile_use_background_image)

    df = pd.DataFrame({'ID': id_usuario,
                       'Usuario': usuario,
                       'Usuario Tela': usuario_tela,
                       'Data_Criacao': data_criacao,
                       'N_Seguidores': n_seguidores,
                       'N_Retweets': n_retweets,
                       'N_Favoritos': total_favoritos,
                       'Verificado': verificado,
                       'Background_Img': tem_background_imagem})

    df_tweets = pd.DataFrame()

    for id_usuario in df['ID']: #para cada usuário pelo pela palavra-chave

        aux = twitter.user_timeline(user_id = id_usuario, count = 16, tweet_mode='extended', include_rts = False)
        # pega os ultimos 16 tweets do usuário sem incluir RT
        # https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline
        
        try:
            tweet0 = aux[0].full_text.replace('\n',' ')
        except:
            tweet0 = " "
        try:
            tweet1 = aux[1].full_text.replace('\n',' ')
        except:
            tweet1 = " "
        try:   
            tweet2 = aux[2].full_text.replace('\n',' ')
        except:
            tweet2 = " "
        try:
            tweet3 = aux[3].full_text.replace('\n',' ')
        except:
            tweet3 = " "
        try:
            tweet4 = aux[4].full_text.replace('\n',' ')
        except:
            tweet4 = " "
        try:
            tweet5 = aux[5].full_text.replace('\n',' ')
        except:
            tweet5 = " "
        try:
            tweet6 = aux[6].full_text.replace('\n',' ')
        except:
            tweet6 = " "
        try:   
            tweet7 = aux[7].full_text.replace('\n',' ')
        except:
            tweet7 = " "
        try:
            tweet8 = aux[8].full_text.replace('\n',' ')
        except:
            tweet8 = " "
        try:
            tweet9 = aux[9].full_text.replace('\n',' ')
        except:
            tweet9 = " "
        try:
            tweet10 = aux[10].full_text.replace('\n',' ')
        except:
            tweet10 = " "
        try:
            tweet11 = aux[11].full_text.replace('\n',' ')
        except:
            tweet11 = " "
        try:
            tweet12 = aux[12].full_text.replace('\n',' ')
        except:
            tweet12 = " "
        try:   
            tweet13 = aux[13].full_text.replace('\n',' ')
        except:
            tweet13 = " "
        try:
            tweet14 = aux[14].full_text.replace('\n',' ')
        except:
            tweet14 = " "
        try:
            tweet15 = aux[15].full_text.replace('\n',' ')
        except:
            tweet15 = " "

        if tweet0 == " " or tweet0 == None or tweet0 == np.nan: 
        #verifica se o primeiro tweet está vazio. Se tiver, não grava as informações desse usuário
            df_aux = pd.DataFrame()
        else:
            df_aux = pd.DataFrame([np.array([str(id_usuario), tweet0, tweet1, tweet2, tweet3, tweet4, 
                                             tweet5, tweet6, tweet7, tweet8, tweet9, tweet10, tweet11, 
                                             tweet12, tweet13, tweet14, tweet15])], 
                                  columns=['ID', 'tweet0', 'tweet1', 'tweet2', 'tweet3', 'tweet4', 
                                           'tweet5', 'tweet6', 'tweet7', 'tweet8', 'tweet9', 'tweet10', 'tweet11', 
                                           'tweet12', 'tweet13', 'tweet14', 'tweet15'])

        df_tweets = df_tweets.append(df_aux, ignore_index = True)

    try:
        base = pd.merge(df, df_tweets, on = 'ID', how = 'left')
    except:
        base = pd.DataFrame()

    con = sqlite3.connect('/content/drive/MyDrive/Twitter Crawler/base_twitter.db') 
    #por fim grava a base de dados em um banco Sqlite
    cur = con.cursor()

    base.to_sql("tweets", con = con, if_exists = 'append', index_label = False)
    
    time.sleep(15)
    